In [3]:
import json
from datasets import load_dataset


In [2]:
from datasets import load_dataset

ds = load_dataset("openai/summarize_from_feedback", "axis")

KeyboardInterrupt: 

In [ ]:
df_val = ds["validation"].to_pandas()

In [ ]:
# unique users
df_val["worker"].drop_duplicates().shape

In [ ]:
# unique documents
df_val["info"].drop_duplicates().shape

# Preprocess dataset

In [ ]:
# preprocess document
df_val =df_val.rename(columns={"worker":"uid"})
df_val["doc_id"] = df_val["info"].apply(lambda x: x["id"])
df_val["doc_text"] = df_val["info"].apply(lambda x: x["post"])
# df_val["site"] = df_val["info"].apply(lambda x: x["site"])
# df_val["subreddit"] = df_val["info"].apply(lambda x: x["subreddit"])

df_val["summary_text"] = df_val["summary"].apply(lambda x: x["text"])
df_val["summary_rating"] = df_val["summary"].apply(lambda x: x["axes"]["overall"])
df_val["summary_model"] = df_val["summary"].apply(lambda x: x["policy"])
df_val = df_val[["doc_id", "doc_text", "uid", "summary_model", "summary_text", "summary_rating"]] 
df_val

# appropriate ratings data to create user summaries

In [ ]:
# pick user max ratings as their own summary
# Assumption: For any document, the highest rated summary by the user can be taken as the summary generated by that user
idx = df_val.groupby(['doc_id', 'uid'])['summary_rating'].idxmax()
df_val.loc[idx].to_csv("./openai_reddit_appropriated_user_summary.csv", index=False)


In [ ]:
df_val.loc[idx].summary_rating

# Skip history 

In [4]:
ds_history = load_dataset("openai/summarize_from_feedback", "comparisons")
df_history = ds_history["validation"].to_pandas()

In [ ]:
# overlap between  
len(set(df_history["worker"].drop_duplicates().values).intersection(set(df_val.loc[idx]["uid"].values)))

In [7]:
df_history

,info,summaries,choice,worker,batch,split,extra
0,"{'id': 't3_52mb8y', 'post': 'This is my first ...",[{'text': ' Just came out of 8 year relationsh...,1,p4Oh7rUGyLe1EpilJFWr9sPDpkO016,batch10,valid1,{'confidence': 6}
1,"{'id': 't3_52mb8y', 'post': 'This is my first ...",[{'text': ' Don't know how to date/flirt/flirt...,1,p4Oh7rUGyLe1EpilJFWr9sPDpkO016,batch10,valid1,{'confidence': 7}
2,"{'id': 't3_52mb8y', 'post': 'This is my first ...",[{'text': ' Don't know how to date/flirt/flirt...,1,p4Oh7rUGyLe1EpilJFWr9sPDpkO016,batch10,valid1,{'confidence': 7}
3,"{'id': 't3_52mb8y', 'post': 'This is my first ...",[{'text': ' Just came out of 8 year relationsh...,0,p4Oh7rUGyLe1EpilJFWr9sPDpkO016,batch10,valid1,{'confidence': 4}
4,"{'id': 't3_52mb8y', 'post': 'This is my first ...",[{'text': ' Don't know how to date/flirt/flirt...,1,p4Oh7rUGyLe1EpilJFWr9sPDpkO016,batch10,valid1,{'confidence': 6}
...,...,...,...,...,...,...,...
86081,"{'id': None, 'post': 'I don't know how to ask ...","[{'text': 'Boyfriend talks very slowly, I feel...",0,soP7Okyp0e1WdYxQC9IldpwnznT6V1,edit_b2_eval_test,valid2,{'confidence': 2}
86082,"{'id': None, 'post': 'I honestly do not care a...",[{'text': 'Medical field looks like its going ...,0,soP7Okyp0e1WdYxQC9IldpwnznT6V1,edit_b2_eval_test,valid1,{'confidence': 2}
86083,"{'id': None, 'post': 'Someone I consider to be...","[{'text': 'Met a guy on reddit, became best fr...",0,soP7Okyp0e1WdYxQC9IldpwnznT6V1,edit_b2_eval_test,valid2,{'confidence': 2}
86084,"{'id': None, 'post': 'Me (17F) and my boyfrien...",[{'text': 'Had a drunken peck with a girl at a...,1,soP7Okyp0e1WdYxQC9IldpwnznT6V1,edit_b2_eval_test,valid2,{'confidence': 8}


## Preprocessing

In [ ]:
ds_history = load_dataset("openai/summarize_from_feedback", "comparisons")

In [70]:
import pandas as pd
def get_history_from_reddit_data(outdir="user_click_history.csv", u_sum_path="./openai_reddit_appropriated_user_summary.csv", rating_threshold=5):
    df_history = ds_history["validation"].to_pandas()
    df_history = df_history.rename(columns={"worker":"uid"})
    df_history["doc_id"] = df_history["info"].apply(lambda x: x["id"])
    df_history["doc_text"] = df_history["info"].apply(lambda x: x["post"])
    df_history["subreddit"] = df_history["info"].apply(lambda x: x["subreddit"])
    
    # filter out non subreddit 
    df_history = df_history[~df_history["subreddit"].isna()]
    
    df_history["confidence"] = df_history["extra"].apply(lambda x: x["confidence"])
    history_candidates = df_history[["doc_id", "uid","doc_text", "confidence"]].groupby(["doc_id","doc_text", "uid"]).aggregate("mean")["confidence"]
    history_candidates = history_candidates.reset_index()
    # history_candidates = history_candidates[history_candidates["confidence"]<= rating_threshold]
    
    # filter out usum pairs
    usum_df = pd.read_csv(u_sum_path)
    usum_pairs = list(usum_df[["doc_id", "uid"]].values)
    usum_pairs = [tuple(x) for x in usum_pairs]
    usum_pairs
    history_candidates["valid_pair"] = history_candidates[["doc_id", "uid"]].apply(lambda x: (x["doc_id"], x["uid"]) not in usum_pairs, axis=1)
    history_candidates = history_candidates[history_candidates["valid_pair"]]
    history_candidates["skip_flag"] = history_candidates["confidence"].apply(lambda x: x <= rating_threshold) 
    history_candidates.value_counts("skip_flag")
    history_candidates = history_candidates[["uid", "doc_id", "doc_text", "skip_flag"]].sort_values("uid")   
    history_candidates.to_csv(outdir, index=False)
    return history_candidates


df = get_history_from_reddit_data(rating_threshold=5)
# print(df)